# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.

%load_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
import dask.dataframe as dd
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score
from sklearn.compose import ColumnTransformer
from  sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from glob import glob
fires_dir = os.path.join('SRC_DIR', "data", "fires")
stock_files = glob(os.path.join(fires_dir, "*.csv"))



In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [3]:
dd_fires = dd.read_csv(
    '../../05_src/data/fires/forestfires.csv',
    header=0,
    names=columns,
    assume_missing=True
)

dd_fires.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7.0,5.0,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0,0.0
1,7.0,4.0,oct,tue,90.6,35.4,669.1,6.7,18.0,33.0,0.9,0.0,0.0
2,7.0,4.0,oct,sat,90.6,43.7,686.9,6.7,14.6,33.0,1.3,0.0,0.0
3,8.0,6.0,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2,0.0
4,8.0,6.0,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [4]:

dd_features = dd_fires.groupby('month').apply(
    lambda df: df.assign(
        temp_lag_1 = df['temp'].shift(1),
        rh_lag_1 = df['rh'].shift(1),
        wind_lag_1 = df['wind'].shift(1),
        rain_lag_1 = df['rain'].shift(1)
    )
)

dd_features

/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/groupby.py:2483: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  meta = self._meta_nonempty.apply(func, *meta_args, **meta_kwargs)
/var/folders/dk/j3ll34kx4233h20p0d_ld2t80000gn/T/ipykernel_1487/1031241098.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_features = dd_fires.groupby('month').apply(


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area,temp_lag_1,rh_lag_1,wind_lag_1,rain_lag_1
npartitions=1,,,,,,,,,,,,,,,,,
,float64,float64,string,string,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [5]:
dd_features = dd_features.dropna()
dp_features = dd_features.compute()

  
X = dp_features.drop(columns = [ 'area'])  # feature column
y = dp_features['area'] # target column


/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/groupby.py:229: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:


numeric_features = dp_features.drop(columns = [ 'month', 'day'])
categorical_features = dp_features[['month', 'day']]
std_scaler = StandardScaler()
scaled_feature_numeric = std_scaler.fit_transform(numeric_features)

In [7]:
onehot = OneHotEncoder()
scaled_feature_categorical_l= onehot.fit_transform(categorical_features).toarray()
X_scaled_l = np.hstack((scaled_feature_numeric, scaled_feature_categorical_l))
y = dp_features['area'].values

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [8]:
qt = QuantileTransformer(output_distribution='normal')
X_quant = qt.fit_transform(numeric_features)
scaled_feature_categorical = onehot.fit(categorical_features )
scaled_feature_categorical

/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (505). n_quantiles is set to n_samples.
  warnings.warn(


OneHotEncoder()

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [9]:
# Pipeline A = preproc1 + baseline
numeric_features = dp_features.drop(columns=['month','day','area']).columns.tolist()
categorical_features = ['month','day']




preproc1 = ColumnTransformer([
    ('numeric', StandardScaler(), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


ridge_pipe_1 = Pipeline([
    ('preprocessing',preproc1),
    ('regressor', Ridge())
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ridge_pipe_1.fit(X_train, y_train)

print("Train R²:", ridge_pipe_1.score(X_train, y_train))
print("Test R²:", ridge_pipe_1.score(X_test, y_test))

Train R²: 0.05849294524745796
Test R²: -0.008846040510090303


In [10]:

# Pipeline B = preproc2 + baseline


preproc2 = ColumnTransformer([
    ('numeric', QuantileTransformer(output_distribution='normal'), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


ridge_pipe_2 = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge())
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ridge_pipe_2.fit(X_train, y_train)





Y_pred_test = ridge_pipe_2.predict(X_test)
test_score = ridge_pipe_2.score(X_test, y_test)

print("Train R²:", ridge_pipe_2.score(X_train, y_train))
print("Test R²:", ridge_pipe_2.score(X_test, y_test))


Train R²: 0.06725955559794927
Test R²: -0.025968769216750287


/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (404). n_quantiles is set to n_samples.
  warnings.warn(


In [11]:
# Pipeline C = preproc1 + advanced
preproc1 = ColumnTransformer([
    ('numeric', StandardScaler(), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


pipe_1 = Pipeline([
    ('preproc', preproc1),
    ('regressor', DecisionTreeRegressor(criterion='squared_error', max_depth=3))
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipe_1.fit(X_train, y_train)
print("Train R²:", pipe_1.score(X_train, y_train))
print("Test R²:", pipe_1.score(X_test, y_test))


Train R²: 0.8430597794174671
Test R²: -0.005722447384470586


In [12]:

# Pipeline D = preproc2 + advanced
preproc2 = ColumnTransformer([
    ('numeric', QuantileTransformer(output_distribution='normal'), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


pipe_2 = Pipeline([
    ('preproc', preproc2),
    ('regressor', DecisionTreeRegressor(criterion='squared_error', max_depth=3))
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipe_2.fit(X_train, y_train)
print("Train R²:", pipe_2.score(X_train, y_train))
print("Test R²:", pipe_2.score(X_test, y_test))




/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (404). n_quantiles is set to n_samples.
  warnings.warn(


Train R²: 0.8430597794174671
Test R²: -0.005722447384470586


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [13]:
# Pipeline A = preproc1 + baseline + GridSearch
numeric_features = dp_features.drop(columns=['month','day','area']).columns.tolist()
categorical_features = ['month','day']




preproc1 = ColumnTransformer([
    ('numeric', StandardScaler(), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


ridge_pipe_1 = Pipeline([
    ('preprocessing',preproc1),
    ('regressor', Ridge())
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0],
    
    }

grid_search1 = GridSearchCV(
    estimator=ridge_pipe_1,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)


grid_search1.fit(X_train, y_train)

res = grid_search1.cv_results_
res = pd.DataFrame(res)
res.columns

print("🏆 Best alpha (preproc2):", grid_search1.best_params_)
print("🏆 Best CV R² (preproc2):", grid_search1.best_score_)


🏆 Best alpha (preproc2): {'regressor__alpha': 100.0}
🏆 Best CV R² (preproc2): -0.3329721915919818


In [14]:

# Pipeline B = preproc2 + baseline + GridSearch
preproc2 = ColumnTransformer([
    ('numeric', QuantileTransformer(output_distribution='normal'), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


ridge_pipe2 = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge())
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


param_grid = {
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0],
    
    }

grid_search2 = GridSearchCV(
    estimator=ridge_pipe2,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

grid_search2.fit(X_train, y_train)

res = grid_search2.cv_results_
res = pd.DataFrame(res)
res.columns


print("🏆 Best alpha (preproc2):", grid_search2.best_params_)
print("🏆 Best CV R² (preproc2):", grid_search2.best_score_)


🏆 Best alpha (preproc2): {'regressor__alpha': 100.0}
🏆 Best CV R² (preproc2): -0.32197583172298294


/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(
/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(
/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(
/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(


In [15]:
# Pipeline C = preproc1 + Advancede + GridSearch
preproc1 = ColumnTransformer([
    ('numeric', StandardScaler(), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


pipe_1 = Pipeline([
    ('preproc', preproc1),
    ('regressor', DecisionTreeRegressor(criterion='squared_error', max_depth=3))
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'regressor__max_depth': [3, 5],
    'regressor__min_samples_leaf': [1, 2]
}



grid_search_1 = GridSearchCV(
    estimator=pipe_1,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_search_1.fit(X_train, y_train)


print("🏆 Best parameters:", grid_search_1.best_params_)
print("🏆 Best CV R²:", grid_search_1.best_score_)




🏆 Best parameters: {'regressor__max_depth': 5, 'regressor__min_samples_leaf': 1}
🏆 Best CV R²: -2.118234235572063


In [16]:

# Pipeline D = preproc2 + advanced + GridSearch



preproc2 = ColumnTransformer([
    ('numeric', QuantileTransformer(output_distribution='normal'), numeric_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])


ridge_pipe2 = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge())
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


param_grid = {
    'regressor__max_depth': [3, 5],
    'regressor__min_samples_leaf': [1, 2]
}
# This gives 2 x 2 = 4 combinations

grid_search_2= GridSearchCV(
    estimator=pipe_2,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)


grid_search_2.fit(X_train, y_train)


print("🏆 Best alpha (preproc2):", grid_search2.best_params_)
print("🏆 Best CV R² (preproc2):", grid_search2.best_score_)



🏆 Best alpha (preproc2): {'regressor__alpha': 100.0}
🏆 Best CV R² (preproc2): -0.32197583172298294


/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(
/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(
/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(
/Users/rahelehmosleh/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (323). n_quantiles is set to n_samples.
  warnings.warn(


# Evaluate

+ Which model has the best performance?

In [18]:
#Since all the CV R² values are negative, their performances are worse than simply predicting the mean of the target. None of the models perform well; in particular, model No. 3 is the worst, with an R² of -2.

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.